# Imports and Initializations

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler
torch.backends.cuda.matmul.allow_tf32 = True
import pandas as pd

num_epochs = 2
lr = 5e-5                                           
batch_size = 1
warmup_steps= 750

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Initialising model

In [2]:
model = GPT2LMHeadModel.from_pretrained("gpt2-large")
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

optimizer = AdamW(model.parameters(), lr=lr)

# Dataset

In [7]:
dataset = pd.read_csv('malayalam_dataset.csv', nrows=100000)['Text']
dataset

0        മറ്റുള്ളവരെ കുറിച്ചു ചിന്തിക്കുന്നത്‌ മറ്റൊരു ...
1        ഓസ്ട്രേലിയൻ സർവീസ് ടീമിനെതിരെ ബാറ്റ് ചെയ്ത ബ്ര...
2                                        പുറത്ത്‌ മഞ്ഞുമഴ.
3                               അതിന് ശേഷം ഇലക്ട്രിക് ബസ്.
4        മലയാളം, കന്നഡ, തമിഴ് തുടങ്ങിയ ഭാഷകളിൽ ശ്രദ്ധേയ...
                               ...                        
99995    സര്‍ക്കാര്‍ ഇക്കാര്യം ഇതുവരെ ഔദ്യോഗികമായി പുറത...
99996                             വാര്‍ത്താ സമ്മേളനത്തില്‍
99997                   എന്ന പേരിനോടു സാമ്യമുള്ള അർദാശിർ. 
99998    ഇത് 27 bhp കരുത്തും 28 Nm torque ഉം പരമാവധി സൃ...
99999    പക്ഷെ ടെസ്റ്റ് ക്രിക്കറ്റ് തികച്ചും വ്യത്യസ്തമ...
Name: Text, Length: 100000, dtype: object

In [8]:
new_dataset = {'input_ids': [], 'attention_mask': []}   
wrong_data = 0
for i,data in enumerate(dataset[:100000]):
    if i%1e5 == 0: print(i)
    try:
        encoded_data = tokenizer(data+'', truncation=True, max_length=768, padding="max_length")
        new_dataset['input_ids'].append(encoded_data['input_ids'])
        new_dataset['attention_mask'].append(encoded_data['attention_mask'])
    except:
        wrong_data += 1
wrong_data

0


0

In [18]:
new_dataset['input_ids'][0], new_dataset['attention_mask'][0]

(tensor([  156,   112,   106,   156,   112,   109,   156,   113,   235,   156,
           112,   109,   156,   113,   223,   156,   112,   111,   156,   113,
           235,   156,   112,   111,   156,   112,   113,   156,   112,   108,
           156,   113,   228,   220,   156,   112,   243,   156,   113,   223,
           156,   112,   109,   156,   112,   123,   156,   112,   248,   156,
           113,   235,   156,   112,   248,   156,   113,   223,   220,   156,
           112,   248,   156,   112,   123,   156,   112,   101,   156,   113,
           235,   156,   112,    97,   156,   112,   123,   156,   112,   243,
           156,   113,   235,   156,   112,   243,   156,   113,   223,   156,
           112,   101,   156,   113,   235,   156,   112,   101,   156,   112,
            97,   156,   113,   235,   447,   234,   220,   156,   112,   106,
           156,   112,   109,   156,   113,   235,   156,   112,   109,   156,
           113,   232,   156,   112,   108,   156,  

In [9]:
new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")
dataloader = DataLoader(new_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)


# Inits

In [11]:
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model = torch.compile(model)
scaler = GradScaler()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# Training loop

In [13]:
save_loc = '/Model_save/test_save.pt'

In [14]:
progress_bar = tqdm(range(num_training_steps-1), desc='Training', unit='steps')
model.train()
ep = 0
prev_avg_train_loss = 1
for epoch in range(num_epochs):
    total_train_loss = 0
    for batch in dataloader:
        batch_data = batch['input_ids'].to(device)
        attention = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(batch_data,
                            labels=batch_data,
                            attention_mask=attention,
                            token_type_ids=None
                            )

            loss = outputs[0]
            batch_loss = loss.item()
            total_train_loss += batch_loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        lr_scheduler.step()
        progress_bar.update(1)

    avg_train_loss = total_train_loss / len(dataloader)
    ep += 1
    print('Epoch:', ep, 'Average training loss =', avg_train_loss)
    if abs(prev_avg_train_loss - avg_train_loss) < 0.0001:
        model.save_pretrained(save_loc)
        tokenizer.save_pretrained(save_loc)
        print("Loss is very small")
        break
    prev_avg_train_loss = avg_train_loss
    model.save_pretrained(save_loc)
    tokenizer.save_pretrained(save_loc)


Training:   0%|          | 0/199999 [00:00<?, ?steps/s]

KeyboardInterrupt: 